## Imports

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 58.3 MB/s 
     |████████████████████████████████| 3.3 MB 50.9 MB/s 
     |████████████████████████████████| 895 kB 72.7 MB/s 
     |████████████████████████████████| 61 kB 616 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# from preprocessing import preprocess
from transformers import DistilBertTokenizer, DistilBertModel, GPT2Tokenizer, GPT2Model
import torch
from torch import nn, optim
import copy
import random
import sklearn.metrics
import tqdm
import pickle
import pandas as pd
import numpy as np
import math

## Helper Functions

In [3]:
def shuffle_data(input_1, input_2, labels):
    shuffled_input_1 = []
    shuffled_input_2 = []
    shuffled_labels      = []
    indices = list(range(len(input_1)))
    random.shuffle(indices)
    for i in indices:
        shuffled_input_1.append(input_1[i])
        shuffled_input_2.append(input_2[i])
        shuffled_labels.append(labels[i])
    return (shuffled_input_1, shuffled_input_2, shuffled_labels)


In [4]:
def train(model, tokenizer, X_1, X_2, Y, learning_rate=0.1, batch_size=8, num_epochs=5):
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  for epoch in range(num_epochs):
      total_loss = 0.0
      (shuffled_input_1, shuffled_input_2, shuffled_labels) = shuffle_data(X_1, X_2, Y)
      for batch in tqdm.notebook.tqdm(range(0, len(X_1), batch_size), leave=False):
          #Randomly shuffle examples in each epoch
          input_1 = shuffled_input_1[batch:(batch + batch_size)]
          input_2 = shuffled_input_2[batch:(batch + batch_size)]
          encoded_input_1 = tokenizer(input_1, return_tensors='pt', is_split_into_words=False, padding=True)
          encoded_input_2 = tokenizer(input_2, return_tensors='pt', is_split_into_words=False, padding=True)
          labels = shuffled_labels[batch:(batch + batch_size)]
          labels_onehot = torch.zeros(len(labels), num_classes).cuda()
          for i in range(len(labels)):
              labels_onehot[i][labels[i]] = 1.0
          model.zero_grad()
          log_probs = model.forward(encoded_input_1, encoded_input_2, train=True)
          # print(log_probs)
          loss_batch = 0
          for idx in range(labels_onehot.shape[0]):
              loss_iteration = torch.neg(log_probs[idx]).dot(labels_onehot[idx])
              loss_batch += loss_iteration
          loss_batch /= labels_onehot.shape[0]
          loss_batch.backward()
          nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          total_loss += loss_batch.detach()
      num_batches = math.ceil(len(X_1) / batch_size)
      print(f"avg loss on epoch {epoch} = {total_loss / num_batches}")

In [5]:
def get_predictions(model, X_1, X_2, batch_size=8):
  all_predictions = np.array([])
  for batch in tqdm.notebook.tqdm(range(0, len(X_1), batch_size), leave=False):
    encoded_input_1 = tokenizer(X_1[batch:batch + batch_size], return_tensors='pt', is_split_into_words=False, padding=True)
    encoded_input_2 = tokenizer(X_2[batch:batch + batch_size], return_tensors='pt', is_split_into_words=False, padding=True)
    log_probs = model.forward(encoded_input_1, encoded_input_2, train=False)
    prediction_batch = torch.argmax(log_probs, dim=1)
    all_predictions = np.concatenate((all_predictions, prediction_batch.cpu().numpy()))
  return all_predictions

In [6]:
def get_predictions_cosine_similarity(model, tokenizer, X_1, X_2, threshold=0.96, batch_size=8):
  all_predictions = np.array([])
  for batch in tqdm.notebook.tqdm(range(0, len(X_1), batch_size), leave=False):
    encoded_input_1 = tokenizer(X_1[batch:batch + batch_size], return_tensors='pt', is_split_into_words=False, padding=True)
    encoded_input_2 = tokenizer(X_2[batch:batch + batch_size], return_tensors='pt', is_split_into_words=False, padding=True)
    pooler_output_1 = model(encoded_input_1['input_ids'].cuda(), encoded_input_1['attention_mask'].cuda()).last_hidden_state[:, 0].cuda()
    pooler_output_2 = model(encoded_input_2['input_ids'].cuda(), encoded_input_2['attention_mask'].cuda()).last_hidden_state[:, 0].cuda()
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    output = cos(pooler_output_1, pooler_output_2).cpu().detach().numpy()
    preds = []
    for i in range(output.shape[0]):
      if output[i] > threshold:
        preds.append(1)
      else:
        preds.append(0)
    all_predictions = np.concatenate((all_predictions, preds))
  return all_predictions

In [7]:
def evaluate(Y, predictions):
  print("Accuracy: {}".format(sklearn.metrics.accuracy_score(Y, predictions)))
  print("F1 score: {}".format(sklearn.metrics.f1_score(Y, predictions)))
  print("Precision: {}".format(sklearn.metrics.precision_score(Y, predictions)))
  print("Recall: {}".format(sklearn.metrics.recall_score(Y, predictions)))
  print("Confusion matrix: \n{}\n".format(sklearn.metrics.confusion_matrix(Y, predictions)))

## Data Loader

In [8]:
# Raw csv
# df = pd.read_csv('train.csv')
# data = [list(df["question1"]), list(df["question2"]), list(df["is_duplicate"])]

# Preprocessed w/o transitivity
# with open('/content/drive/MyDrive/processed_data_wo_transitive.pkl', 'rb') as f:
#    data = pickle.load(f)

# Preprocessed w/ transitivity
# Finalized transitivity based on its superiority
with open('/content/drive/MyDrive/processed_data_2.pkl', 'rb') as f:
   data = pickle.load(f)

print("Original data has {} question pairs".format(len(data[0])))

Original data has 577240 question pairs


In [9]:
size = len(data[0])
dataset = data[:][:size]
train_ratio = 0.8
indices = list(range(size))
random.shuffle(indices)
train_indices = indices[:int(size*train_ratio)]
test_indices = indices[int(size*train_ratio):]
train_dataset = [[dataset[i][j] for j in train_indices] for i in range(len(dataset))]
test_dataset = [[dataset[i][j] for j in test_indices] for i in range(len(dataset))]

train_input_1 = [" ".join(train_dataset[0][i]) for i in range(len(train_dataset[0]))]
train_input_2 = [" ".join(train_dataset[1][i]) for i in range(len(train_dataset[1]))]
# train_input_1 = ["".join(train_dataset[0][i]) for i in range(len(train_dataset[0]))]
# train_input_2 = ["".join(train_dataset[1][i]) for i in range(len(train_dataset[1]))]
train_Y = train_dataset[2]
print(len(train_input_1), len(train_input_2), len(train_Y))
num_classes = 2

test_input_1 = [" ".join(test_dataset[0][i]) for i in range(len(test_dataset[0]))]
test_input_2 = [" ".join(test_dataset[1][i]) for i in range(len(test_dataset[1]))]
test_Y = test_dataset[2]
print(len(test_input_1), len(test_input_2), len(test_Y))
num_classes = 2

461792 461792 461792
115448 115448 115448


## Fine Tune Model

In [10]:
class SimilarityModelFineTuneGPT(nn.Module):
    def __init__(self, dropout_rate=0.25):
        super(SimilarityModelFineTuneGPT, self).__init__()
        self.gpt = GPT2Model.from_pretrained("distilgpt2").cuda()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.feedforward_1 = nn.Linear(768*2, 300).cuda()
        self.non_lin_1 = nn.PReLU().cuda()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.feedforward_2 = nn.Linear(300, 300).cuda()
        self.non_lin_2 = nn.PReLU().cuda()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.feedforward_3 = nn.Linear(300, 2).cuda()
        self.log_softmax = nn.LogSoftmax(dim=0).cuda()

    def forward(self, encoded_input_1, encoded_input_2, train=False):
        if train:
          self.train()
        else:
          self.eval()
        pooler_output_1 = self.gpt(input_ids=encoded_input_1['input_ids'].cuda(), attention_mask=encoded_input_1['attention_mask'].cuda()).last_hidden_state[:, 0].cuda()
        pooler_output_2 = self.gpt(input_ids=encoded_input_2['input_ids'].cuda(), attention_mask=encoded_input_2['attention_mask'].cuda()).last_hidden_state[:, 0].cuda()
        # print(pooler_output_1.keys())
        concatenated_output = torch.cat([pooler_output_1, pooler_output_2], axis=1).cuda()
        f1 = self.dropout(self.non_lin_1(self.feedforward_1(concatenated_output)))
        f2 = self.dropout(self.non_lin_2(self.feedforward_2(f1)))
        return self.log_softmax(self.feedforward_3(f2))

## Config

In [11]:
dropout_rate = 0.25
batch_size = 64
learning_rate = 0.0001
num_epochs = 5

## Train

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
print("Training fine tune model")
fine_tune_model = SimilarityModelFineTuneGPT()
train(fine_tune_model, tokenizer, train_input_1, train_input_2, train_Y, learning_rate=learning_rate, num_epochs=num_epochs, batch_size=batch_size)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Training fine tune model


...

avg loss on epoch 2 = 4.096896648406982


  0%|          | 0/7216 [00:00<?, ?it/s]

avg loss on epoch 3 = 4.092881679534912


  0%|          | 0/7216 [00:00<?, ?it/s]

avg loss on epoch 4 = 4.0893964767456055


In [13]:
with open("gpt-ffn-fine-tune-pickle.pkl", 'wb') as f:
  pickle.dump(fine_tune_model, f)

In [14]:
torch.save(fine_tune_model.state_dict(), 'gpt-ffn-fine-tune-state-dict.pt')

## Eval

In [15]:
print("Evaluating fine tune model on train dataset")
predictions = get_predictions(fine_tune_model, train_input_1, train_input_2, batch_size=batch_size)
evaluate(train_Y, predictions)

print("Evaluating fine tune bert model on test dataset")
predictions = get_predictions(fine_tune_model, test_input_1, test_input_2, batch_size=batch_size)
evaluate(test_Y, predictions)

Evaluating fine tune model on train dataset


  0%|          | 0/7216 [00:00<?, ?it/s]

Accuracy: 0.6515898932852886
F1 score: 0.5827409587779924
Precision: 0.5063068097321802
Recall: 0.6863560833760965
Confusion matrix: 
[[188548 109552]
 [ 51341 112351]]

Evaluating fine tune bert model on test dataset


  0%|          | 0/1804 [00:00<?, ?it/s]

Accuracy: 0.6389976439609175
F1 score: 0.5675986927426466
Precision: 0.493006993006993
Recall: 0.6687856042639544
Confusion matrix: 
[[46417 28130]
 [13547 27354]]



## Cosine Similarity Baseline (Discontinued) 

In [ ]:
# Stopped working on this after evaluating a baseline

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
vanilla = DistilBertModel.from_pretrained("distilbert-base-uncased").cuda()
print("Evaluating vanilla model on train dataset")
print("NOTE: For vanilla model with cosine based similarity, train-test split doesn't matter\n")
predictions = get_predictions_cosine_similarity(vanilla, tokenizer, train_input_1, train_input_2, 0.96)
evaluate(train_Y, predictions)

print("Evaluating vanilla model on test dataset")
print("NOTE: For vanilla model with cosine based similarity, train-test split doesn't matter\n")
predictions = get_predictions_cosine_similarity(vanilla, tokenizer, test_input_1, test_input_2, 0.96)
evaluate(test_Y, predictions)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating vanilla model on train dataset
NOTE: For vanilla model with cosine based similarity, train-test split doesn't matter



  0%|          | 0/1000 [00:00<?, ?it/s]

Accuracy: 0.598375
F1 score: 0.6064911206368647
Precision: 0.4746932515337423
Recall: 0.8396066463207867
Confusion matrix: 
[[2311 2740]
 [ 473 2476]]

Evaluating vanilla model on test dataset
NOTE: For vanilla model with cosine based similarity, train-test split doesn't matter



  0%|          | 0/250 [00:00<?, ?it/s]

Accuracy: 0.6035
F1 score: 0.6170931916948333
Precision: 0.4881588999236058
Recall: 0.8385826771653543
Confusion matrix: 
[[568 670]
 [123 639]]



## Static BERT Model (Discontinued)

In [ ]:
# Stopped training this after superiority of finetuning BERT was realized

class SimilarityModelStaticBert(nn.Module):
    def __init__(self):
        super(SimilarityModelStaticBert, self).__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased").cuda()
        for param in self.bert.parameters():
            param.requires_grad = False
        self.feedforward_1 = nn.Linear(768*2, 300).cuda()
        self.non_lin_1 = nn.PReLU().cuda()
        self.feedforward_2 = nn.Linear(300, 300).cuda()
        self.non_lin_2 = nn.PReLU().cuda()
        self.feedforward_3 = nn.Linear(300, 2).cuda()
        self.log_softmax = nn.LogSoftmax(dim=0).cuda()

    def forward(self, encoded_input_1, encoded_input_2, train=False):
        if train:
          self.train()
        else:
          self.eval()
        pooler_output_1 = self.bert(encoded_input_1['input_ids'].cuda(), encoded_input_1['attention_mask'].cuda()).last_hidden_state[:, 0].cuda()
        pooler_output_2 = self.bert(encoded_input_2['input_ids'].cuda(), encoded_input_2['attention_mask'].cuda()).last_hidden_state[:, 0].cuda()
        # print(pooler_output_1)
        concatenated_output = torch.cat([pooler_output_1, pooler_output_2], axis=1).cuda()
        f1 = self.non_lin_1(self.feedforward_1(concatenated_output))
        f2 = self.non_lin_2(self.feedforward_2(f1))
        return self.log_softmax(self.feedforward_3(f2))


tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
print("Training static bert model")
static_bert_model = SimilarityModelStaticBert()
train(static_bert_model, tokenizer, train_input_1, train_input_2, train_Y, learning_rate=0.0001, num_epochs=10, batch_size=32)

Training static bert model


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/250 [00:00<?, ?it/s]

avg loss on epoch 0 = 0.10724116116762161


  0%|          | 0/250 [00:00<?, ?it/s]

...

avg loss on epoch 7 = 0.10562307387590408


  0%|          | 0/250 [00:00<?, ?it/s]

avg loss on epoch 8 = 0.10542453825473785


  0%|          | 0/250 [00:00<?, ?it/s]

avg loss on epoch 9 = 0.10536253452301025


In [ ]:
print("Evaluating static bert model on train dataset")
predictions = get_predictions(static_bert_model, train_input_1, train_input_2, batch_size=32)
evaluate(train_Y, predictions)

print("Evaluating static bert model on test dataset")
predictions = get_predictions(static_bert_model, test_input_1, test_input_2, batch_size=32)
evaluate(test_Y, predictions)

Evaluating static bert model on train dataset


  0%|          | 0/250 [00:00<?, ?it/s]

Accuracy: 0.70625
F1 score: 0.6392385630948726
Precision: 0.5840112201963534
Recall: 0.7060020345879959
Confusion matrix: 
[[3568 1483]
 [ 867 2082]]

Evaluating static bert model on test dataset


  0%|          | 0/63 [00:00<?, ?it/s]

Accuracy: 0.6835
F1 score: 0.6165960024227742
Precision: 0.5725534308211474
Recall: 0.6679790026246719
Confusion matrix: 
[[858 380]
 [253 509]]

